INSTALACIÓN DE LIBRERIAS

In [ ]:
import pandas as pd

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_selector
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import classification_report
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import RandomOverSampler
from imblearn.combine import SMOTETomek
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import multiprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay



Un tumor puede ser canceroso o benigno. Un tumor canceroso es maligno, lo que significa que puede crecer y diseminarse a otras partes del cuerpo. Un tumor benigno significa que el tumor puede crecer, pero no se diseminará.

CARGAMOS EL DATASET

In [ ]:
df = pd.read_csv('./data.csv',sep=',',encoding='latin1')
df


SE ELIMINA LA VARIABLE ID PORQUE NO APORTA AL MODELO

In [ ]:
df.set_index('id', inplace=True)
df = df.drop(['Unnamed: 32'], axis=1)
df.head()

REVISAMOS QUE NO HAYA VALORES NULOS

In [ ]:
df.info()

Convertimos la varible que deseamos predecir en numeros.
Para lo cual se asignara 1 para Maligno y 0 para Benigno

In [ ]:
df.diagnosis = df.diagnosis.replace('M', 1)
df.diagnosis = df.diagnosis.replace('B', 0)
df.diagnosis.value_counts()

In [ ]:
# Dimensiones del dataset
# ==============================================================================
df.shape

In [ ]:
# Número de datos ausentes por variable
# ==============================================================================
df.isna().sum().sort_values()

In [ ]:
count_classes = pd.value_counts(df['diagnosis'], sort = True)
count_classes.plot(kind = 'bar', rot=0)
LABELS=['Benigno','Maligno']
plt.xticks(range(2), LABELS)
plt.title("Frecuencia de observaciones")
plt.xlabel("Diagnostico")
plt.ylabel("Numero de observaciones");

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


# Calculamos la matriz de correlacion
corr = df.corr()

# plot
plt.figure(figsize=(20, 20))
sns.heatmap(corr,
        xticklabels=corr.columns,
        yticklabels=corr.columns,
        cmap='YlGnBu', linewidths=.5,annot=True)

División train y test

In [ ]:
# Reparto de datos en train y test
# ==============================================================================
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
                                        df.drop('diagnosis', axis = 'columns'),
                                        df['diagnosis'],
                                        train_size   = 0.2,
                                        random_state = 1234,
                                        shuffle      = True
                                    )

In [ ]:
print("BASE COMPLETA")
print(df.diagnosis.value_counts())
print("-----------------------")
print("Partición de test")
print(y_test.value_counts())
print("-----------------------")
print("Partición de train")
print(y_train.value_counts())

PREDICCION

In [ ]:
def plot_confusion_matrix(X, y, model):
    y_pred = model.predict(X)
    cm = confusion_matrix(y, y_pred, labels=model.classes_)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                                  display_labels=model.classes_)
    disp.plot()

    plt.show()

In [ ]:
# Sin bucle plantilla basica
def evaluate_model(X_train, y_train, X_test, y_test, model):
    print(f"Error training: {(1 - model.score(X_train, y_train)) * 100} %")
    print(f"Error test: {(1 - model.score(X_test, y_test)) * 100} %")
    print("")
    print("")
    print("")


    print("Training confusion matrix")
    plot_confusion_matrix(X_train, y_train, model)
    print("Test confusion matrix")
    plot_confusion_matrix(X_test, y_test, model)

1.- MODELO KNN

In [ ]:

model = KNeighborsClassifier(n_neighbors=5)
model.fit(X_train, y_train)

evaluate_model(X_train, y_train, X_test, y_test, model)

2.- MODELO BOOSTING

Es una técnica de aprendizaje automático que combina varios clasificadores o modelos débiles para crear un modelo sólido. La idea detrás del impulso es entrenar una secuencia de modelos, cada uno de los cuales se enfoca en los ejemplos que previamente fueron mal clasificados por el modelo anterior.

In [ ]:

model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.2,
    max_depth=1, random_state=0)

model = model.fit(X_train, y_train)

evaluate_model(X_train, y_train, X_test, y_test, model)

3.- MODELO SVC

In [ ]:

model = SVC()

model = model.fit(X_train, y_train)
evaluate_model(X_train, y_train, X_test, y_test, model)

CONCLUSION:

En el modelo KNN tenemos un error de train de 7.07% y de test de 7.68%, en cuanto a la matriz de confusión tenemos que este modelo predijo correctamente 142 casos de cancer benignos y 279 malignos, los falsos negativos que nos da el modelo es de 27 casos.

El modelo Boosting tenemos un error de train de 0.0% y de test de 5.26%, en cuanto a la matriz de confusión tenemos que este modelo predijo correctamente 283 casos de cancer benignos y 149 malignos, los falsos negativos que nos da el modelo es de 20 casos.

El modelo Boosting tenemos un error de train de 9.73% y de test de 10.96%, en cuanto a la matriz de confusión tenemos que este modelo predijo correctamente 284 casos de cancer benignos y 122 malignos, los falsos negativos que nos da el modelo es de 47 casos.

El modelo Boosting es el que mejor predice la base de datos del cáncer. Debido a que otorga solo 20 casos falsos negativos.